In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from trainlib.FileCollection import FileCollection
from trainlib.config import Config
from trainlib.ConfigFileHandler import ConfigFileHandler
from trainlib.ConfigFileUtils import ConfigFileUtils
import trainlib.cuts as cuts
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import copy
import re
from scipy import interpolate
import scipy.integrate as integrate
import pickle
import os

Welcome to JupyROOT 6.10/09


In [3]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
#candidate_branches = ["PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "ZZMass", "nExtraZ", "Z1Mass", "Z2Mass", "Z1Pt", "Z2Pt", "ZZMassErr", "ZZPt", "ZZEta", "ZZPhi", "Z1Flav", "Z2Flav", "costhetastar", "helphi", "helcosthetaZ1", "helcosthetaZ2", "phistarZ1", "phistarZ2", "xi", "xistar"]
candidate_branches = ["PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "ZZMass", "nExtraZ", "Z1Mass", "Z2Mass", "Z1Pt", "Z2Pt", "ZZMassErr", "ZZPt", "ZZEta", "ZZPhi", "Z1Flav", "Z2Flav"]
MELA_branches = ["D_VBF2j_ggH_ME", "D_VBF1j_ggH_ME", "D_WHh_ggH_ME", "D_ZHh_ggH_ME", "D_WHh_ZHh_ME", "D_VBF2j_WHh_ME", "D_VBF2j_ZHh_ME"]
#list_branches = ["Jet", "Lep", "ExtraLep"]
list_branches = ["Jet", "ExtraLep"]
pt_limits = [30.0, 0.0, 0.0]

In [5]:
allbranches = ["JetPt", "JetEta", "JetPhi", "LepPt", "LepEta", "LepPhi", "ExtraLepPt", "ExtraLepEta", "ExtraLepPhi"] + candidate_branches + MELA_branches + ["LHEAssociatedParticleId", "GenAssocLep1Id", "GenAssocLep2Id", "training_weight"]

In [6]:
#MC_path = "/data_CMS/cms/wind/CJLST_NTuples_randomizeda/"
MC_path = "/data_CMS/cms/wind/CJLST_NTuples/"

In [7]:
# these are the cuts without any m4l restriction imposed
def WHhadr0j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] == 0

def WHhadr01j_cut(row):
    return cuts.WHhadr_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def WHhadr1j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] == 1

def WHhadr2j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] >= 2

def ZHhadr0j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] == 0

def ZHhadr01j_cut(row):
    return cuts.ZHhadr_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def ZHhadr1j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] == 1

def ZHhadr2j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] >= 2

def mZZ0j_cut(row):
    return row["nCleanedJetsPt30"] == 0

def mZZ01j_cut(row):
    return (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def mZZ1j_cut(row):
    return row["nCleanedJetsPt30"] == 1

def mZZ2j_cut(row):
    return row["nCleanedJetsPt30"] >= 2

In [8]:
collections = {"VBF2j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ2j_cut},
            "VBF1j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ1j_cut},
            "VBF0j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ0j_cut},
            "VBF01j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ01j_cut},
            "VBF": {MC_path + "VBFH125/ZZ4lAnalysis.root": cuts.mZZ_cut},
            "ggH2j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ2j_cut},
            "ggH1j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ1j_cut},
            "ggH0j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ0j_cut},
            "ggH01j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ01j_cut},
            "ggH" : {MC_path + "ggH125/ZZ4lAnalysis.root": cuts.mZZ_cut},
            "WHh2j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr2j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr2j_cut},
            "WHh1j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr1j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr1j_cut},
            "WHh0j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr0j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr0j_cut},
            "WHh": {MC_path + "WplusH125/ZZ4lAnalysis.root": cuts.WHhadr_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": cuts.WHhadr_cut},
            "WHh01j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr01j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr01j_cut},
            "WHl": {MC_path + "WplusH125/ZZ4lAnalysis.root": cuts.WHlept_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": cuts.WHlept_cut},
            "ZHh2j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr2j_cut},
            "ZHh1j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr1j_cut},
            "ZHh01j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr01j_cut},
            "ZHh0j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr0j_cut},
            "ZHh": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHhadr_cut},
            "ZHl": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHlept_cut},
            "ttHh": {MC_path + "ttH125/ZZ4lAnalysis.root": cuts.ttHhadr_cut},
            "ttHl": {MC_path + "ttH125/ZZ4lAnalysis.root": cuts.ttHlept_cut},
            "ZHMET": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHMET_cut}
          }

In [9]:
# all the model combinations for which neural networks are currently trained
discriminant_pairs = [("VBF2j", "ggH2j"), ("VBF1j", "ggH1j"), ("VBF0j", "ggH0j"), ("WHh2j", "ggH2j"), 
                     ("WHh1j", "ggH1j"), ("WHh0j", "ggH0j"), ("ZHh2j", "ggH2j"), ("ZHh1j", "ggH1j"), 
                      ("ZHh0j", "ggH0j"), ("WHh2j", "ZHh2j"), ("WHh01j", "ZHh01j"), ("VBF2j", "WHh2j"),
                     ("VBF1j", "WHh1j"), ("VBF0j", "WHh0j"), ("VBF2j", "ZHh2j"), ("VBF1j", "ZHh1j"), 
                      ("VBF0j", "ZHh0h"), ("WHl", "ggH"), ("WHl", "VBF"), ("WHl", "WHh"), ("WHl", "ZHh"),
                     ("WHl", "ZHl"), ("WHl", "ZHMET"), ("WHl", "ttHh"), ("WHl", "ttHl"), ("ZHh", "ZHl"),
                     ("ZHh", "ZHMET"), ("ZHh", "ttHh"), ("ZHh", "ttHl"), ("ZHl", "ggH"), ("ZHl", "VBF"),
                     ("ZHl", "WHh"), ("ZHl", "ZHMET"), ("ZHl", "ttHh"), ("ZHl", "ttHl"), ("ZHMET", "ggH"),
                     ("ZHMET", "VBF"), ("ZHMET", "WHh"), ("ZHMET", "ttHh"), ("ZHMET", "ttHl"), ("ttHh", "ggH"),
                      ("ttHh", "VBF"), ("ttHh", "WHh"), ("ttHh", "ttHl"), ("ttHl", "ggH"), ("ttHl", "VBF"),
                     ("ttHl", "WHh")]

In [10]:
def extract_order(df, col_basename, sorted_column, columns, order):
    def get_index(row, order, col_basename, sorted_column):
        sorted_column = row[col_basename + sorted_column]
        if order >= len(sorted_column):
            return -1
        else:
            return np.flipud(np.argsort(sorted_column))[order]
    
    index_column = pd.DataFrame(df.transform(lambda row: get_index(row, order, col_basename, sorted_column), axis = 1, raw = True))
    index_column.columns = ["index"]
    df_temp = pd.concat([index_column, df], axis = 1)
    
    def get_element(row, column_name):
        if row["index"] == -1:
            return 0
        else:
            return row[column_name][row["index"]]
        
    extracted_cols = pd.DataFrame()
    for column in columns:
        extracted_col = pd.DataFrame(df_temp.transform(lambda row: get_element(row, col_basename + column), axis = 1, raw = True))
        extracted_col.columns = [col_basename + column + "(" + col_basename + "Pt|" + str(order) + ")"]
        extracted_cols = pd.concat([extracted_cols, extracted_col], axis = 1)
        
    return extracted_cols

In [11]:
def prepare_data(df, col_basenames, sorted_column, columns, orders, pt_limits):
    all_extracted = pd.DataFrame()
    for col_basename, pt_limit in zip(col_basenames, pt_limits):
        for order in orders:
            extracted = extract_order(df, col_basename, sorted_column, columns, order)
            mask = extracted[col_basename + "Pt(" + col_basename + "Pt|" + str(order) + ")"] < pt_limit
            extracted[mask] = 0.0

            all_extracted = pd.concat([all_extracted, extracted], axis = 1)
            
    return all_extracted

In [12]:
def get_data(H1_coll, H0_coll, read_branches, input_branches, list_branches, pt_limits):
    H1_df = H1_coll.get_data(read_branches, 0.0, 1.0)
    H0_df = H0_coll.get_data(read_branches, 0.0, 1.0)
    
    H1_list_df = prepare_data(H1_df, list_branches, "Pt", ["Pt", "Eta", "Phi"], range(4), pt_limits)
    H0_list_df = prepare_data(H0_df, list_branches, "Pt", ["Pt", "Eta", "Phi"], range(4), pt_limits)
    
    list_branches_unrolled = H1_list_df.columns
            
    H1_df = pd.concat([H1_df, H1_list_df], axis = 1)
    H0_df = pd.concat([H0_df, H0_list_df], axis = 1)        
    
    complete_input_branches = np.concatenate([input_branches, list_branches_unrolled])
            
    H1_df = H1_df[complete_input_branches]
    H0_df = H0_df[complete_input_branches]
    
    return H1_df, H0_df

In [13]:
def get_data_dmatrix(H1_coll, H0_coll, read_branches, input_branches, list_branches, pt_limits):
    H1_df, H0_df = get_data(H1_coll, H0_coll, read_branches, input_branches, list_branches, pt_limits)
    
    complete_input_branches = H1_df.columns
    print "number of input variables: " + str(len(complete_input_branches))
    print "final list of inputs: " + str(complete_input_branches)
    
    # try with the same weights as used later in the neural network training, to balance out some (very)
    # unbalanced datasets
    H1_class_weight = 1.0 + float(len(H0_df)) / float(len(H1_df))
    H0_class_weight = 1.0 + float(len(H1_df)) / float(len(H0_df))
    
    print "using class weights: " + str(H1_class_weight) + " (H1), " + str(H0_class_weight) + " (H0)"
    
    H1_weights = np.full(len(H1_df), H1_class_weight)
    H0_weights = np.full(len(H0_df), H0_class_weight)
    
    H1_data = H1_df.as_matrix()
    H0_data = H0_df.as_matrix()
    H1_target = np.ones(np.shape(H1_data)[0])
    H0_target = np.zeros(np.shape(H0_data)[0])
    
    target = np.concatenate([H1_target, H0_target])
    data = np.concatenate([H1_data, H0_data])
    weights = np.concatenate([H1_weights, H0_weights])
    
    dmatrix = xgb.DMatrix(data, label = target, feature_names = complete_input_branches, weight = weights)
    
    return dmatrix

In [14]:
def get_feature_correlation(source, corr_branches, mandatory_branches, optional_branches, list_branches, pt_limits):    
    coll = FileCollection(collections[source], 0.0, 0.5)
    
    input_branches = [branch for branch in mandatory_branches]
    
    for optional_branch in optional_branches:
        if "0j" in source and ("0j" in optional_branch):
            input_branches.append(optional_branch)
            
        if "1j" in source and ("1j" in optional_branch):
            input_branches.append(optional_branch)
            
        if "2j" in source and ("2j" in optional_branch):
            input_branches.append(optional_branch)

    df, _ = get_data(coll, coll, allbranches, input_branches, list_branches, pt_limits)

    df = df[corr_branches]
    
    fig = plt.figure(figsize=(10,15))
    ax = fig.add_subplot(111)
    
    cax = ax.matshow(df.corr(), vmin = -1.0, vmax = 1.0, cmap = "RdBu")
    
    fig.colorbar(cax)
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    ax.set_yticklabels([''] + corr_branches)
    ax.set_xticklabels([''] + corr_branches, rotation = 'vertical')
    
    return fig

In [15]:
def get_interpolating_function(data, bins):
    bin_centers = [np.mean([bins[i], bins[i + 1]]) for i in range(len(bins) - 1)]
    intf = interpolate.interp1d(bin_centers, data, kind = "linear")
    interpolated_function = lambda x: intf(x) if x > bin_centers[0] and x < bin_centers[-1] else 0
    
    return interpolated_function

In [16]:
def get_binned_data(df, branch):
    data = df[branch].as_matrix()
    weights = df["training_weight"].as_matrix()
    
    # set the bin width
    q75, q25 = np.percentile(data, [75, 25])
    bin_width = max(2 * (q75 - q25) / len(data)**0.33, 0.005)

    data_max = np.max(data)
    data_min = np.min(data)
    bins = np.arange(data_min, data_max + bin_width, bin_width)
    
    weights = weights / (np.sum(weights) * bin_width)
    
    hist = np.histogram(data, bins = bins, weights = weights)
    return hist

In [17]:
def get_feature_importance_list_separation(disc_pair, mandatory_branches, optional_branches, list_branches, pt_limits):
    H1_name = disc_pair[0]
    H0_name = disc_pair[1]
    
    input_branches = [branch for branch in mandatory_branches]
    
    for optional_branch in optional_branches:
        if ("0j" in H1_name or "0j" in H0_name) and ("0j" in optional_branch):
            input_branches.append(optional_branch)  
        elif ("1j" in H1_name or "1j" in H0_name) and ("1j" in optional_branch):
            input_branches.append(optional_branch)
        elif ("2j" in H1_name or "2j" in H0_name) and not ("1j" in optional_branch):
            input_branches.append(optional_branch)
            
    # needed to build the histograms
    input_branches.append("training_weight")
    
    H1_coll = FileCollection(collections[H1_name], 0.0, 0.5)
    H0_coll = FileCollection(collections[H0_name], 0.0, 0.5)
    
    H1_df, H0_df = get_data(H1_coll, H0_coll, allbranches, input_branches, list_branches, pt_limits)
    
    available_branches = H1_df.columns
    
    implist = {}
    for branch in available_branches:
                
        if "training_weight" not in branch:
            data_H1, bins_H1 = get_binned_data(H1_df, branch)
            data_H0, bins_H0 = get_binned_data(H0_df, branch)

            if(len(data_H1) > 0):
                H1_func = get_interpolating_function(data_H1, bins_H1)
                H0_func = get_interpolating_function(data_H0, bins_H0)

                # compute the separation in this branch
                global_min = np.min(np.concatenate([bins_H0, bins_H1]))
                global_max = np.max(np.concatenate([bins_H0, bins_H1]))

                separation_func = lambda x: (H1_func(x) + H0_func(x)) * (H1_func(x) - H0_func(x))**2

                sep = integrate.quad(separation_func, global_min, global_max)[0]
            else:
                sep = 0.0
                
            print "separation for " + branch + " = " + str(sep)
            implist[branch] = sep
        
    # normalize the importance list
    impsum = sum([val for key, val in implist.iteritems()])
    for key in implist.keys():
        implist[key] /= impsum
        
    return None, None, implist

In [18]:
def get_feature_importance_list_BDT(disc_pair, mandatory_branches, optional_branches, list_branches, pt_limits):
    H1_name = disc_pair[0]
    H0_name = disc_pair[1]
    
    # first assemble the list of branches that can serve as input: it will *always* contain the mandatory branches,
    # and *can* contain some of the optional branches, if the name of the categories allows it
    input_branches = [branch for branch in mandatory_branches]
    
    for optional_branch in optional_branches:
        if ("0j" in H1_name or "0j" in H0_name) and ("0j" in optional_branch):
            input_branches.append(optional_branch)  
        elif ("1j" in H1_name or "1j" in H0_name) and ("1j" in optional_branch):
            input_branches.append(optional_branch)
        elif ("2j" in H1_name or "2j" in H0_name) and not ("1j" in optional_branch):
            input_branches.append(optional_branch)
            
        # the fully inclusive categories (i.e. those with NO "xxj" in their name, can not use MELA, since there may
        # be events with low number of jets contained)
    
    # get the training data for the BDT ...
    H1_coll_train = FileCollection(collections[H1_name], 0.0, 0.5)
    H0_coll_train = FileCollection(collections[H0_name], 0.0, 0.5)
    
    dtrain = get_data_dmatrix(H1_coll_train, H0_coll_train, allbranches, input_branches, list_branches, pt_limits)
    
    # ... and the validation data as well
    H1_coll_val = FileCollection(collections[H1_name], 0.5, 1.0)
    H0_coll_val = FileCollection(collections[H0_name], 0.5, 1.0)
    dval = get_data_dmatrix(H1_coll_val, H0_coll_val, allbranches, input_branches, list_branches, pt_limits)
    
    evallist = [(dtrain, 'train'), (dval, 'eval')]
    
    # perform the training
    # try different tree depths and choose the one that gives the best RMSE (i.e. avoid too deep trees to start with)
        
    params = {'eta': 0.01, 'silent': 1, 'gamma': 0.5, 'objective': 'binary:logistic'}
    params['nthread'] = 4
    params['eval_metric'] = 'rmse'
    max_num_rounds = 2000
    
    best_loss = 1e6
    best_imp = None
    best_params = None
    for tree_depth in range(1,8):
        params['max_depth'] = tree_depth
        
        bst = xgb.train(params, dtrain, max_num_rounds, evals = evallist, early_stopping_rounds = 10, verbose_eval = False)
    
        pred = bst.predict(dval)
        cur_loss = np.sqrt(mean_squared_error(pred, dval.get_label()))
        cur_imp = bst.get_fscore()

        print "for max_depth = " + str(params['max_depth']) + ": loss = " + str(cur_loss)
        
        if cur_loss < best_loss:
            best_loss = cur_loss
            best_imp = copy.copy(cur_imp)
            best_params = copy.copy(params)
            
    # normalize the usage score w.r.t. the total score (i.e. sum of all individuals)
    score_sum = sum([val for key, val in best_imp.iteritems()])
    used_variables = {key: val / float(score_sum) for key, val in sorted(best_imp.iteritems(), key = lambda x: x[1], reverse = True)}           
    return best_params, dtrain.feature_names, used_variables

In [19]:
def get_histogram(df, branch, label):
    data = df[branch].as_matrix()
    weights = df["training_weight"].as_matrix()
    
    # set the bin width
    q75, q25 = np.percentile(data, [75, 25])
    bin_width = max(2 * (q75 - q25) / len(data)**0.33, 0.005)

    data_max = np.max(data)
    data_min = np.min(data)
    bins = np.arange(data_min, data_max + bin_width, bin_width)
    
    weights = weights / (np.sum(weights) * bin_width)
    
    fig = plt.hist(data, bins = bins, weights = weights, alpha = 0.5, label = label)
    return fig

In [20]:
def plot_branch(disc_pair, branch, start_fraction = 0.0, end_fraction = 1.0):
    H1_name = disc_pair[0]
    H0_name = disc_pair[1]
    
    # get the training data for the BDT ...
    H1_coll = FileCollection(collections[H1_name], start_fraction, end_fraction)
    H0_coll = FileCollection(collections[H0_name], start_fraction, end_fraction)
    
    H1_df, H0_df = get_data(H1_coll, H0_coll, allbranches, allbranches, list_branches, pt_limits)
    
    plt.figure()
    H1_hist = get_histogram(H1_df, branch, H1_name)
    H0_hist = get_histogram(H0_df, branch, H0_name)
    
    plt.legend(loc = 'upper right')
    plt.show()

In [21]:
def plot_variables(discs):
    plotframe = pd.DataFrame()
    
    for disc in discs:
        _, _, implist = get_feature_importance_list_BDT(disc, candidate_branches, MELA_branches, list_branches, pt_limits)
        
        # cut the list to select only the 95% most important variables
        cutimplist = {key: val for key, val in implist.iteritems() if val > 0.00}
        curframe = pd.DataFrame(cutimplist, index = [len(plotframe)])
        
        plotframe = pd.concat([plotframe, curframe])
        
    plotframe = plotframe.fillna(0.0)
    
    print plotframe
    print "number of pre-selected input variables = " + str(len(plotframe.columns))
    
    # start the plotting
    parameters = plotframe.columns
    plotdata = np.transpose(plotframe.as_matrix())
    
    plt.close('all')
    fig = plt.figure(figsize=(10,15))
    ax = fig.add_subplot(111)
    cax = ax.matshow(plotdata, cmap = 'Blues')
    
    # make axis labels
    disclabels = []
    for disc in discs:
        if "0j" in disc[0] or "0j" in disc[1]:
            disclabels.append('D_' + re.sub('0j', '', disc[0]) + "_" + re.sub('0j', '', disc[1]) + "_0j")
        elif "01j" in disc[0] or "01j" in disc[1]:
            disclabels.append('D_' + re.sub('01j', '', disc[0]) + "_" + re.sub('01j', '', disc[1]) + "_01j")
        elif "1j" in disc[0] or "1j" in disc[1]:
            disclabels.append('D_' + re.sub('1j', '', disc[0]) + "_" + re.sub('1j', '', disc[1]) + "_1j")
        elif "2j" in disc[0] or "2j" in disc[1]:
            disclabels.append('D_' + re.sub('2j', '', disc[0]) + "_" + re.sub('2j', '', disc[1]) + "_2j")
        else:
            disclabels.append('D_' + disc[0] + "_" + disc[1] + "_2j")
            
    disclabels = np.concatenate([[''], np.array(disclabels)])
    parameters = np.concatenate([[''], np.array(parameters)])
        
    ax.set_xticklabels(disclabels, rotation = 'vertical')
    ax.set_yticklabels(parameters)
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    # sort the used variables according to their importance
    sorted_implist = []
    for key, val in sorted(cutimplist.iteritems(), key = lambda x: x[1], reverse = True):
        sorted_implist.append((key, val))
    
    return fig, sorted_implist

In [22]:
def append_variables(confhandler, impdict, threshold_fscore):
    confhandler.new_section(impdict["discriminant"])
    cur_sec = confhandler.get_section(impdict["discriminant"])

    periodic_inputs = []
    nonperiodic_inputs = []
    for key, val in impdict.iteritems():
        if val[0] > threshold_fscore and key is not "discriminant":
            if "phi" in key or "Phi" in key:
                periodic_inputs.append(key)
            else:
                nonperiodic_inputs.append(key)
    cur_sec["nonperiodic_columns"] = ConfigFileUtils.serialize_list(nonperiodic_inputs, lambda x: x)
    cur_sec["periodic_columns"] = ConfigFileUtils.serialize_list(periodic_inputs, lambda x: x)

In [23]:
def convert_varname(raw):
    raw = raw.replace('(', '[')
    raw = raw.replace(')', ']')
    return raw

In [24]:
df = pd.DataFrame()

In [25]:
out_dir = "/data_CMS/cms/wind/InputConfigurations/"
out_path = os.path.join(out_dir, "exclusive_99_fullmassrange.conf")
threshold_fscore = 0.01

In [26]:
confhandler = ConfigFileHandler()

In [27]:
%%capture
fig, implist = plot_variables([("VBF2j", "ggH2j")])

In [28]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_ggH_2j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [29]:
append_variables(confhandler, impdict, threshold_fscore)

In [30]:
implist

[('D_VBF2j_ggH_ME', 0.06387977760718357),
 ('JetPt(JetPt|0)', 0.06278642380254496),
 ('JetPt(JetPt|1)', 0.05280666247935422),
 ('PFMET', 0.051317840277293136),
 ('ZZPt', 0.051061948961313885),
 ('JetEta(JetPt|1)', 0.04326889524740038),
 ('JetEta(JetPt|0)', 0.042803638309256285),
 ('JetPhi(JetPt|1)', 0.042780375462349085),
 ('ZZEta', 0.041873124432968103),
 ('Z1Pt', 0.04101239909740154),
 ('JetEta(JetPt|2)', 0.04066345639379347),
 ('Z2Mass', 0.0398027310582269),
 ('ZZPhi', 0.03694140088864075),
 ('JetPhi(JetPt|0)', 0.036452881103589456),
 ('ZZMass', 0.03638309256286784),
 ('D_WHh_ZHh_ME', 0.03570847000255891),
 ('ZZMassErr', 0.0356386814618373),
 ('Z2Pt', 0.0356386814618373),
 ('Z1Mass', 0.03501058459534278),
 ('D_WHh_ggH_ME', 0.02970665550050015),
 ('D_ZHh_ggH_ME', 0.029404238490706492),
 ('JetPt(JetPt|2)', 0.029055295787098426),
 ('D_VBF2j_WHh_ME', 0.02289064135668923),
 ('D_VBF2j_ZHh_ME', 0.022774327122153208),
 ('JetPhi(JetPt|2)', 0.01544653034638379),
 ('JetPhi(JetPt|3)', 0.0048386

In [31]:
%%capture
fig, implist = plot_variables([("VBF1j", "ggH1j")])

In [32]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_ggH_1j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [33]:
append_variables(confhandler, impdict, threshold_fscore)

In [34]:
implist

[('JetEta(JetPt|0)', 0.14584154513204572),
 ('ZZEta', 0.0856873822975518),
 ('D_VBF1j_ggH_ME', 0.08505233653046029),
 ('JetPt(JetPt|0)', 0.08100118249901458),
 ('ZZPt', 0.07684053781807033),
 ('PFMET', 0.07561424254368677),
 ('Z1Pt', 0.0657162878290194),
 ('JetPhi(JetPt|0)', 0.05505189856786231),
 ('ZZMass', 0.05496430604826348),
 ('ZZMassErr', 0.05467963035956729),
 ('Z1Mass', 0.05386939955327815),
 ('ZZPhi', 0.0536942145140805),
 ('Z2Pt', 0.04979634739193273),
 ('Z2Mass', 0.04907370910524241),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0044672184995401395),
 ('Z1Flav', 0.003021941926159506),
 ('Z2Flav', 0.002364998029168309),
 ('ExtraLepPt(ExtraLepPt|0)', 0.0016204616125782857),
 ('ExtraLepPhi(ExtraLepPt|0)', 0.0011168046248850349),
 ('ExtraLepEta(ExtraLepPt|0)', 0.0003722682082950116),
 ('nExtraLep', 0.00015328690929794595)]

In [35]:
%%capture
fig, implist = plot_variables([("VBF0j", "ggH0j")])

In [36]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_ggH_0j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [37]:
append_variables(confhandler, impdict, threshold_fscore)

In [38]:
implist

[('ZZPt', 0.1606582032113947),
 ('PFMET', 0.11248728270004865),
 ('ZZMass', 0.10903702393064095),
 ('ZZEta', 0.09970363161852523),
 ('Z1Pt', 0.09545715928694652),
 ('Z2Pt', 0.08966249391781307),
 ('ZZPhi', 0.08276197637899765),
 ('Z1Mass', 0.0816561242093157),
 ('Z2Mass', 0.08130225151501748),
 ('ZZMassErr', 0.07670190648914053),
 ('Z1Flav', 0.004644579112664219),
 ('Z2Flav', 0.0030963860751094794),
 ('nExtraLep', 0.0009731499093201221),
 ('ExtraLepEta(ExtraLepPt|0)', 0.0008846817357455655),
 ('ExtraLepPhi(ExtraLepPt|0)', 0.0005308090414473393),
 ('ExtraLepPt(ExtraLepPt|0)', 0.00044234086787278275)]

In [39]:
%%capture
fig, implist = plot_variables([("WHh2j", "ggH2j")])

In [40]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHh_ggH_2j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [41]:
append_variables(confhandler, impdict, threshold_fscore)

In [42]:
implist

[('JetPt(JetPt|0)', 0.05882962195753495),
 ('JetPt(JetPt|1)', 0.05427239772138788),
 ('D_VBF2j_ggH_ME', 0.0524287933713102),
 ('ZZEta', 0.05178663904712584),
 ('PFMET', 0.04996374935266701),
 ('ZZMass', 0.04967374417400311),
 ('JetEta(JetPt|0)', 0.04859658208182289),
 ('ZZPt', 0.048513723459347485),
 ('D_WHh_ggH_ME', 0.04493008803728638),
 ('Z1Pt', 0.040932159502848266),
 ('JetEta(JetPt|2)', 0.03950284826514759),
 ('ZZPhi', 0.03554634904194718),
 ('JetEta(JetPt|1)', 0.035380631796996374),
 ('JetPhi(JetPt|0)', 0.03469704816157431),
 ('Z2Pt', 0.034137752459865355),
 ('ZZMassErr', 0.03393060590367685),
 ('D_WHh_ZHh_ME', 0.033516312791299845),
 ('Z1Mass', 0.032749870533402385),
 ('Z2Mass', 0.03250129466597618),
 ('D_ZHh_ggH_ME', 0.032128430864836875),
 ('JetPhi(JetPt|1)', 0.03132055929570171),
 ('JetPhi(JetPt|2)', 0.024919730709476954),
 ('JetPt(JetPt|2)', 0.02392542723977214),
 ('D_VBF2j_ZHh_ME', 0.022931123770067324),
 ('D_VBF2j_WHh_ME', 0.01615743138270326),
 ('JetEta(JetPt|3)', 0.00901

In [43]:
%%capture
fig, implist = plot_variables([("WHh1j", "ggH1j")])

In [44]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHh_ggH_1j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [45]:
append_variables(confhandler, impdict, threshold_fscore)

In [46]:
implist

[('JetEta(JetPt|0)', 0.13845729267283013),
 ('ZZEta', 0.11585483648406238),
 ('JetPt(JetPt|0)', 0.09405271146681385),
 ('D_VBF1j_ggH_ME', 0.07870843107492756),
 ('ZZPt', 0.07639023044018214),
 ('PFMET', 0.0639161032151235),
 ('ZZMassErr', 0.061018352421691735),
 ('ZZMass', 0.05900372567959156),
 ('Z1Pt', 0.05362218849178971),
 ('ZZPhi', 0.053456602732165036),
 ('Z2Mass', 0.051717952256105974),
 ('Z2Pt', 0.051717952256105974),
 ('Z1Mass', 0.04570166965640955),
 ('JetPhi(JetPt|0)', 0.044404581206016286),
 ('Z2Flav', 0.004139643990616807),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0025665792741824205),
 ('ExtraLepPt(ExtraLepPt|0)', 0.0020146267421001793),
 ('Z1Flav', 0.0016282599696426107),
 ('ExtraLepEta(ExtraLepPt|0)', 0.0008831240513315855),
 ('nExtraLep', 0.0007175382917069132),
 ('ExtraLepPhi(ExtraLepPt|0)', 2.7597626604112046e-05)]

In [47]:
%%capture
fig, implist = plot_variables([("WHh0j", "ggH0j")])

In [48]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHh_ggH_0j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [49]:
append_variables(confhandler, impdict, threshold_fscore)

In [50]:
implist

[('ZZPt', 0.16273490603758498),
 ('PFMET', 0.11168865787018525),
 ('ZZMass', 0.10655737704918032),
 ('ZZEta', 0.10582433693189391),
 ('ZZMassErr', 0.09429561508729842),
 ('ZZPhi', 0.08869785419165667),
 ('Z2Mass', 0.08483273357323737),
 ('Z2Pt', 0.07963481274157004),
 ('Z1Pt', 0.07730241236838598),
 ('Z1Mass', 0.07570305211248834),
 ('Z1Flav', 0.004998000799680128),
 ('Z2Flav', 0.0035319205651072904),
 ('ExtraLepPhi(ExtraLepPt|0)', 0.0032653605224576836),
 ('nExtraLep', 0.0008663201386112222),
 ('ExtraLepEta(ExtraLepPt|0)', 6.66400106624017e-05)]

In [51]:
%%capture
fig, implist = plot_variables([("ZHh2j", "ggH2j")])

In [52]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHh_ggH_2j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [53]:
append_variables(confhandler, impdict, threshold_fscore)

In [54]:
implist

[('JetPt(JetPt|0)', 0.0652191591445068),
 ('JetPt(JetPt|1)', 0.05354964625518419),
 ('D_ZHh_ggH_ME', 0.05139464910140685),
 ('ZZPt', 0.051110026835813616),
 ('JetEta(JetPt|0)', 0.05037814101000244),
 ('D_VBF2j_ggH_ME', 0.0499308774497845),
 ('Z1Pt', 0.04826380417988127),
 ('ZZEta', 0.046840692851915104),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.04106692689273807),
 ('ZZPhi', 0.04086362527445719),
 ('JetEta(JetPt|2)', 0.03956249491745954),
 ('JetEta(JetPt|1)', 0.03874928844433602),
 ('Z2Mass', 0.03854598682605514),
 ('JetPhi(JetPt|0)', 0.03427665284215663),
 ('PFMET', 0.03374806863462633),
 ('ZZMass', 0.033260144750752216),
 ('D_WHh_ggH_ME', 0.030617223713100758),
 ('Z1Mass', 0.03045458241847605),
 ('Z2Pt', 0.02935675367975929),
 ('D_WHh_ZHh_ME', 0.026876473936732537),
 ('D_VBF2j_WHh_ME', 0.025006099048548426),
 ('JetPhi(JetPt|2)', 0.02427421322273725),
 ('ZZMassErr', 0.023461006749613728),
 ('JetPt(JetPt|2)', 0.023379686102301376),
 ('JetPhi(JetPt|1)', 0.021712612832398146),
 ('D_VBF2j_ZH

In [55]:
%%capture
fig, implist = plot_variables([("ZHh1j", "ggH1j")])

In [56]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHh_ggH_1j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [57]:
append_variables(confhandler, impdict, threshold_fscore)

In [58]:
implist

[('JetEta(JetPt|0)', 0.11710013282488539),
 ('ZZEta', 0.11401516774497622),
 ('JetPt(JetPt|0)', 0.09486267620720683),
 ('D_VBF1j_ggH_ME', 0.08800719825185312),
 ('ZZPhi', 0.0754102575088907),
 ('ZZPt', 0.07112558378679464),
 ('ZZMass', 0.06559835468529071),
 ('Z2Mass', 0.056729080080551865),
 ('Z1Pt', 0.05437250953339903),
 ('PFMET', 0.05253009983289773),
 ('JetPhi(JetPt|0)', 0.048502506534127424),
 ('Z1Mass', 0.046960023994172845),
 ('ZZMassErr', 0.0463601696730794),
 ('Z2Pt', 0.04408929260036848),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.012468400531299542),
 ('ExtraLepPt(ExtraLepPt|0)', 0.003084965079909165),
 ('nExtraLep', 0.0026993444449205195),
 ('Z2Flav', 0.002185183598268992),
 ('ExtraLepPhi(ExtraLepPt|0)', 0.0016281760143965038),
 ('Z1Flav', 0.001242555379407858),
 ('ExtraLepEta(ExtraLepPt|0)', 0.001028321693303055)]

In [59]:
%%capture
fig, implist = plot_variables([("ZHh0j", "ggH0j")])

In [60]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHh_ggH_0j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [61]:
append_variables(confhandler, impdict, threshold_fscore)

In [62]:
implist

[('ZZPt', 0.14670854600183222),
 ('Z2Mass', 0.11870174060986782),
 ('ZZMassErr', 0.10914801727522576),
 ('Z2Pt', 0.10522182960345504),
 ('ZZPhi', 0.10208087946603847),
 ('Z1Mass', 0.08663787462374035),
 ('Z1Pt', 0.08624525585656327),
 ('ZZEta', 0.08297343279675436),
 ('PFMET', 0.07420494699646643),
 ('ZZMass', 0.07302709069493522),
 ('Z1Flav', 0.007852375343541421),
 ('ExtraLepPt(ExtraLepPt|0)', 0.004973171050909567),
 ('Z2Flav', 0.0014396021463159272),
 ('ExtraLepPhi(ExtraLepPt|0)', 0.0007852375343541421)]

In [63]:
%%capture
fig, implist = plot_variables([("WHh2j", "ZHh2j")])

In [64]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHh_ZHh_2j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [65]:
append_variables(confhandler, impdict, threshold_fscore)

In [66]:
implist

[('D_WHh_ZHh_ME', 0.0660503845882436),
 ('JetPhi(JetPt|1)', 0.05440688730505963),
 ('PFMET', 0.051937054548020604),
 ('ZZMass', 0.0467151224331381),
 ('Z2Mass', 0.04495095617811023),
 ('ZZMassErr', 0.04219885682026674),
 ('JetEta(JetPt|0)', 0.04219885682026674),
 ('D_WHh_ggH_ME', 0.041704890268858937),
 ('JetPt(JetPt|1)', 0.04001129066403218),
 ('Z1Mass', 0.03987015736362995),
 ('JetPt(JetPt|0)', 0.03923505751181992),
 ('Z2Pt', 0.03768259120739539),
 ('ZZEta', 0.037259191306188696),
 ('JetPhi(JetPt|0)', 0.03690635805518312),
 ('Z1Pt', 0.03669465810457977),
 ('JetEta(JetPt|1)', 0.035565591701361934),
 ('ZZPhi', 0.03168442594030062),
 ('JetPhi(JetPt|2)', 0.030978759438289465),
 ('D_ZHh_ggH_ME', 0.03076705948768612),
 ('ZZPt', 0.02780326017923929),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.027732693529038177),
 ('JetPt(JetPt|2)', 0.026533060475619222),
 ('D_VBF2j_ggH_ME', 0.025403994072401383),
 ('D_VBF2j_WHh_ME', 0.02505116082139581),
 ('D_VBF2j_ZHh_ME', 0.02194622821254675),
 ('JetEta(JetPt

In [67]:
%%capture
fig, implist = plot_variables([("WHh1j", "ZHh1j")])

In [68]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHh_ZHh_1j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [69]:
append_variables(confhandler, impdict, threshold_fscore)

In [70]:
implist

[('ZZMass', 0.11099712761592122),
 ('Z2Mass', 0.0883258104226508),
 ('Z2Pt', 0.08627410750923266),
 ('ZZMassErr', 0.08165777595404186),
 ('ZZPhi', 0.0720147722609766),
 ('PFMET', 0.07129667624128026),
 ('Z1Mass', 0.06955272876487485),
 ('JetPt(JetPt|0)', 0.0613459171112023),
 ('ZZPt', 0.05970455478046779),
 ('D_VBF1j_ggH_ME', 0.05857611817808781),
 ('JetPhi(JetPt|0)', 0.05642183011899877),
 ('ZZEta', 0.05365203118588428),
 ('JetEta(JetPt|0)', 0.05313910545752975),
 ('Z1Pt', 0.050984817398440706),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.018362741075092327),
 ('Z2Flav', 0.0029749692244562987),
 ('Z1Flav', 0.0025646286417726713),
 ('ExtraLepPt(ExtraLepPt|0)', 0.002154288059089044)]

In [71]:
%%capture
fig, implist = plot_variables([("WHh0j", "ZHh0j")])

In [72]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHh_ZHh_0j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [73]:
append_variables(confhandler, impdict, threshold_fscore)

In [74]:
implist

[('PFMET', 0.18597560975609756),
 ('ZZPhi', 0.1524390243902439),
 ('ZZPt', 0.1402439024390244),
 ('ZZEta', 0.125),
 ('Z1Pt', 0.11890243902439024),
 ('Z2Pt', 0.07926829268292683),
 ('Z2Mass', 0.07317073170731707),
 ('Z1Mass', 0.06402439024390244),
 ('ZZMassErr', 0.04573170731707317),
 ('nExtraLep', 0.01524390243902439)]

In [75]:
%%capture
fig, implist = plot_variables([("VBF2j", "WHh2j")])

In [76]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_WHh_2j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [77]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

[('JetEta(JetPt|2)', 0.08227812000912617),
 ('D_VBF2j_ggH_ME', 0.07805726671229751),
 ('JetEta(JetPt|0)', 0.06202943189596167),
 ('JetPt(JetPt|2)', 0.053416609628108604),
 ('ZZMass', 0.047256445357061375),
 ('JetEta(JetPt|1)', 0.04637234770704997),
 ('ZZEta', 0.0453171343828428),
 ('D_WHh_ggH_ME', 0.04200889801505818),
 ('JetPt(JetPt|0)', 0.04121035820214465),
 ('PFMET', 0.039812913529545975),
 ('D_WHh_ZHh_ME', 0.03684690851015286),
 ('ZZPt', 0.035706137348847825),
 ('JetPt(JetPt|1)', 0.03553502167465206),
 ('Z2Mass', 0.03259753593429158),
 ('ZZPhi', 0.03171343828428017),
 ('Z1Pt', 0.031399726214921285),
 ('D_ZHh_ggH_ME', 0.03094341775039927),
 ('Z1Mass', 0.02803445128907141),
 ('Z2Pt', 0.027663700661647275),
 ('D_VBF2j_ZHh_ME', 0.02726443075519051),
 ('JetPhi(JetPt|0)', 0.022872461784166097),
 ('ZZMassErr', 0.02255874971480721),
 ('JetPhi(JetPt|1)', 0.019507186858316223),
 ('D_VBF2j_WHh_ME', 0.01691193246634725),
 ('JetPhi(JetPt|2)', 0.01676933607118412),
 ('JetEta(JetPt|3)', 0.011464

In [ ]:
%%capture
fig, implist = plot_variables([("VBF1j", "WHh1j")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_WHh_1j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("VBF0j", "WHh0j")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_WHh_0j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("VBF2j", "ZHh2j")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_ZHh_2j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("VBF1j", "ZHh1j")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_ZHh_1j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("VBF0j", "ZHh0j")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_VBF_ZHh_0j_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "ggH")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_ggH_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "VBF")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_VBF_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "WHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_WHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "ZHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_ZHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "ZHl")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_ZHl_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "ZHMET")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_ZHMET_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "ttHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_ttHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("WHl", "ttHl")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_WHl_ttHl_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHh", "ZHl")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHh_ZHl_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHh", "ZHMET")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHh_ZHMET_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHh", "ttHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHh_ttHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHh", "ttHl")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHh_ttHl_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHl", "ggH")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHl_ggH_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHl", "VBF")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHl_VBF_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHl", "WHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHl_WHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHl", "ZHMET")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHl_ZHMET_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHl", "ttHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHl_ttHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHl", "ttHl")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHl_ttHl_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHMET", "ggH")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHMET_ggH_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHMET", "VBF")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHMET_VBF_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHMET", "WHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHMET_WHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHMET", "ttHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHMET_ttHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ZHMET", "ttHl")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ZHMET_ttHl_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ttHh", "ggH")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ttHh_ggH_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ttHh", "VBF")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ttHh_VBF_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ttHh", "WHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ttHh_WHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ttHh", "ttHl")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ttHh_ttHl_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ttHl", "ggH")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ttHl_ggH_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ttHl", "VBF")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ttHl_VBF_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
%%capture
fig, implist = plot_variables([("ttHl", "WHh")])

In [ ]:
impdict = {convert_varname(entry[0]): [entry[1]] for entry in implist}
impdict["discriminant"] = "D_ttHl_WHh_ML"
df = df.append(pd.DataFrame.from_dict(impdict))

In [ ]:
append_variables(confhandler, impdict, threshold_fscore)

In [ ]:
implist

In [ ]:
# save the variable configuration
confhandler.save_configuration(out_path)

In [ ]:
df = df.fillna(0.0)

In [ ]:
df.to_csv("input_parameters_table.csv")

In [ ]:
# now plot the data contained in the table to have a global picture of the relevant input variables
datacol_labels = [col for col in df.columns.tolist() if col != "discriminant"]
variable_data = df[datacol_labels].as_matrix().transpose()
datacol_labels = np.concatenate([[''], np.array(datacol_labels)])

In [ ]:
discriminant_labels = np.concatenate([[''], df["discriminant"].as_matrix()])

In [ ]:
fig = plt.figure(figsize = (15, 10))
ax = fig.add_subplot(111)
cax = ax.matshow(variable_data, interpolation = 'nearest', cmap = 'Blues', vmin = np.min(variable_data), vmax = np.max(variable_data))
ax.set_xticklabels(discriminant_labels, rotation = 'vertical')
ax.set_yticklabels(datacol_labels)
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

In [ ]:
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "input_variables_exclusive_fullmassrange.pdf"))